
# Optimal power flow model in Julia

In [ ]:
import Pkg

# activate parent environment
Pkg.activate(normpath(joinpath(@__DIR__, ".")))
Pkg.resolve()
Pkg.instantiate()
Pkg.status()

using CSV
using DataFrames
using Dates
using TimeZones
using CairoMakie
using JuMP
import JSON
import MathOptInterface as MOI

using Revise
using OptHP

## Load data

In [ ]:
network = CSV.read("data/network.csv", DataFrame)
first(network, 5)

In [ ]:
connections = CSV.read("data/user_connect.csv", DataFrame; delim=";")

# convert PV (str) to Float64
connections.PV = parse.(Float64, replace.(connections.PV, "," => "."))

first(connections, 5)

In [ ]:
loads_real = CSV.read("data/UserPower.csv", DataFrame)
loads_real.time = DateTime.(loads_real.time, "m/d/yyyy H:M p")

loads_reactive = CSV.read("data/UserReactivePower.csv", DataFrame)
loads_reactive.time = DateTime.(loads_reactive.time, "m/d/yyyy H:M p")

# Filter for rows where the date is February 1, 2024
date = Date(2024, 2, 1)
loads_real = loads_real[Date.(loads_real.time) .== date, 3:end] .* 10
loads_reactive = loads_reactive[Date.(loads_reactive.time) .== date, 3:end] .* 10
first(loads_real, 5)

## Load state-space matrices

In [ ]:
id = "H14"

A_d = Matrix(CSV.read("data/$(id)/$(id)_15min_A_exact.csv", DataFrame))
B_d = Matrix(CSV.read("data/$(id)/$(id)_15min_B_exact.csv", DataFrame))

H14_meta = Dict(
    "A" => A_d,
    "B" => B_d
)

meta = Dict(
    "H14" => H14_meta
)

## Read weather data

In [ ]:
weather_df = CSV.read("data/weather/weather.csv", DataFrame)
select!(weather_df, Not([:interpolated]))
rename!(weather_df, Dict("P_solar" => "Φ_s", "T_ambient" => "T_a"))

dt_format = "yyyy-mm-ddTHH:MM:SS.sss+zzzz"
weather_df.timestamp = ZonedDateTime.(String.(weather_df.timestamp), dt_format)
weather_df = interpolate_data(weather_df, 15)

# select a particular day
date = Date(2024, 4, 1)
weather_df = weather_df[Date.(weather_df.timestamp) .== date, :]

# add electricity price [€/kWh]
weather_df.λ_e .= 0.21

# convert Φ_s from W to kW
weather_df.Φ_s .= weather_df.Φ_s ./ 1000

println("Weather data shape: ", size(weather_df))
first(weather_df, 5)

In [ ]:
# plot the weather data
fig = Figure(; size = (1000, 600))

ax1 = Axis(fig[1, 1], xlabel="Time", ylabel="Solar Irradiance [W/m²]")
ax2 = Axis(fig[2, 1], xlabel="Time", ylabel="Ambient Temperature [°C]")

lines!(ax1, DateTime.(weather_df.timestamp), weather_df.Φ_s, color=:blue)
lines!(ax2, DateTime.(weather_df.timestamp), weather_df.T_a, color=:red)

fig

## Plot user power in kW

In [ ]:
# plot user power data
fig = Figure(; size = (1000, 600))

for (i,user) in enumerate([50,54,56,76])

    ax = Axis(fig[i, 1], xlabel="Time", ylabel="Power [kW]")
    lines!(ax, loads_real[:, "$user"], label="Real", color=:blue)
    lines!(ax, loads_reactive[:, "$user"], label="Reactive", color=:red)
    axislegend(ax)
end

fig

## Construct GEC

In [ ]:
model = GEC(network=network, 
            connections=connections, 
            loads_real=loads_real, 
            loads_reactive=loads_reactive,   
            weather=weather_df,    
            meta = meta,
            silent=false 
)

## Test some outputs

In [ ]:
# plot using Makie
fig = Figure(; size = (1000, 600))
ax = Axis(fig[1, 1], xlabel = "Time [hours]", 
    ylabel = "Power [kW]", 
    title = "Power flow",
    xticks = (1:4:97, string.(0:1:24))
)

# index of slack bus (transformer)
SB = argmin(value.(model[:P]).axes[2])
P_trafo = Matrix(value.(model[:P]; result=1))[:, SB] .* -1E3


# lines!(ax, sol[:P], color = :blue, label = "Transformer", linestyle = :so
scatterlines!(ax, P_trafo, color = :blue, label = "P trafo", linewidth = 2)
axislegend(ax, position = :lb)

# display
fig
